In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __init__ import *
import numpy as np
from export.saver import read_pickle
from helpers.layout import Layout
from helpers.graph_helpers import *
from helpers.helpers import sort_and_group_objects
from typing import Dict
from domains.domain import Domain
from helpers.directions import get_axis, get_opposite_axis, Direction
from placement2.arrange import *
from copy import copy, deepcopy

In [3]:
layout: Layout = read_pickle("1001_amber_c_ag")
layout

Layout(shapes={'m_bed': <POLYGON ((2.24 8.87, 2.24 11.32, 0.11 11.32, 0.11 8.87, 2.24 8.87))>, 'bed1_closet': <POLYGON ((4.18 8.42, 4.18 8.79, 3.03 8.79, 3.03 8.42, 4.18 8.42))>, 'transit': <POLYGON ((2.89 7.91, 2.89 8.79, 1.73 8.79, 1.73 7.91, 2.89 7.91))>, 'm_closet': <POLYGON ((1.58 7.94, 1.58 8.82, 0.11 8.82, 0.11 7.94, 1.58 7.94))>, 'laundry': <POLYGON ((4.18 7.25, 4.18 8.34, 3.03 8.34, 3.03 7.25, 4.18 7.25))>, 'den': <POLYGON ((4.18 5.12, 4.18 7.17, 2.37 7.17, 2.37 5.12, 4.18 5.12))>, 'kitchen': <POLYGON ((1.58 2.6, 1.58 5.27, 0.11 5.27, 0.11 2.6, 1.58 2.6))>, 'living': <POLYGON ((3.27 0, 3.27 2.42, 0.11 2.42, 0.11 0, 3.27 0))>, 'dining': <POLYGON ((3.27 2.42, 3.27 5.02, 1.73 5.02, 1.73 2.42, 3.27 2.42))>, 'den_closet': <POLYGON ((4.18 4.66, 4.18 5.02, 3.52 5.02, 3.52 4.66, 4.18 4.66))>, 'm_bath': <POLYGON ((1.58 6.49, 1.58 7.73, 0 7.73, 0 6.49, 1.58 6.49))>, 'bath': <POLYGON ((1.61 5.43, 1.61 6.36, 0.03 6.36, 0.03 5.43, 1.61 5.43))>, 'linen': <POLYGON ((2.94 7.25, 2.94 7.64, 2.3

In [9]:
arr, *_ = initialize_arr(layout)

In [10]:
arr

array([['m_bed', 'bed1'],
       ['', ''],
       ['', ''],
       ['', ''],
       ['', ''],
       ['', '']], dtype=object)

In [ ]:
get_next_row(0, layout.graph, arr, layout.domains)

0
1
2
3


array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', 'dining', 'den', ''],
       ['kitchen', 'living', 'den_closet', ''],
       ['', '', '', '']], dtype=object)